In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.6 MB/s 

### Setup a client

In [2]:
import chromadb
client = chromadb.Client()

In [3]:
client.get_version()

'1.3.0'

### Setup collection

In [4]:
from chromadb.utils import embedding_functions

In [5]:
# create collection with default embedding
collection = client.create_collection(
    name="stocks",
    embedding_function= embedding_functions.DefaultEmbeddingFunction(),
    get_or_create = True
)

In [6]:
print(collection)

Collection(name=stocks)


### Get Stock Companies Profiles

1. Download the list of 500 companies that make up the S&P500, which we will do from this page (link).

2. For each company, extract the company’s main description from Wikipedia. We will do this with the summary function from the wikipedia library.


In [7]:
!pip install lxml wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=835da95dabc1279c1bc7ffa65d916322fce60225311517d706c05ac0c708d69a
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [8]:
import pandas as pd
import wikipedia

In [9]:
def get_wikipedia_summary(name):
     try:
         summary = wikipedia.summary(name + 'company')
     except:
         summary = None
     finally:
         return summary


### Get data from wikipedia

In [10]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

df = pd.read_html(url, attrs = {"id": "constituents"})[0]
df['company_summary'] = df['Security'].apply(lambda x: get_wikipedia_summary(x))

df.head()

HTTPError: HTTP Error 403: Forbidden

In [11]:
import pandas as pd
import requests

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Send request with a browser-like header
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get(url, headers=headers)
response.raise_for_status()  # raise error if failed

# Read the HTML from response
df_list = pd.read_html(response.text, attrs={"id": "constituents"})
df = df_list[0]  # the first table is what we need

df.head()


/tmp/ipython-input-1377870582.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(response.text, attrs={"id": "constituents"})


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB


In [13]:
df['company_summary'] = df['Security'].apply(lambda x: get_wikipedia_summary(x))

### Clean the data
1. A list with the documents (the summaries of the texts).

2. A list with the ids that we will create.

3. A list of dictionaries with the metadata we want to upload per document. This metadata will include filtering information that may be of interest to us, such as the Sector, the sub-sector, the date the company was founded or the date it was added to the S&P500.

In [15]:
cleaned_df = (
    df
    .query("~company_summary.isnull()")
    .assign(
        date_founded = lambda x: x['Founded'].str.replace('\(.*\)|/.*', '', regex = True).astype(int),
        date_added = lambda x: x['Date added']
    )
    [['Security', 'GICS Sector', 'GICS Sub-Industry', 'date_founded', 'date_added', 'company_summary']]
    .dropna()
    .reset_index(drop=True)
)

documents = cleaned_df['company_summary'].tolist()
ids = cleaned_df.index.astype(str).tolist()
metadata = cleaned_df.drop('company_summary', axis = 1).to_dict(orient = 'records')

<>:5: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\('
/tmp/ipython-input-895699535.py:5: SyntaxWarning: invalid escape sequence '\('
  date_founded = lambda x: x['Founded'].str.replace('\(.*\)|/.*', '', regex = True).astype(int),


In [16]:
documents

["Adobe Inc. (  ə-DOH-bee), formerly Adobe Systems Incorporated, is an American multinational computer software company based in San Jose, California. It offers a wide range of programs from web design tools, photo manipulation and vector creation, through to video/audio editing, mobile app development, print layout and animation software.\nIt has historically specialized in software for the creation and publication of a wide range of content, including graphics, photography, illustration, animation, multimedia/video, motion pictures, and print. Its flagship products include Adobe Photoshop image editing software; Adobe Illustrator vector-based illustration software; Adobe Acrobat Reader and the Portable Document Format (PDF); and a host of tools primarily for audio-visual content creation, editing and publishing. Adobe offered a bundled solution of its products named Adobe Creative Suite, which evolved into a subscription-based offering named Adobe Creative Cloud. The company also exp

In [17]:
ids

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47']

In [18]:
metadata

[{'Security': 'Adobe Inc.',
  'GICS Sector': 'Information Technology',
  'GICS Sub-Industry': 'Application Software',
  'date_founded': 1982,
  'date_added': '1997-05-05'},
 {'Security': 'Alphabet Inc. (Class A)',
  'GICS Sector': 'Communication Services',
  'GICS Sub-Industry': 'Interactive Media & Services',
  'date_founded': 1998,
  'date_added': '2006-04-03'},
 {'Security': 'Alphabet Inc. (Class C)',
  'GICS Sector': 'Communication Services',
  'GICS Sub-Industry': 'Interactive Media & Services',
  'date_founded': 1998,
  'date_added': '2014-04-03'},
 {'Security': 'Amazon',
  'GICS Sector': 'Consumer Discretionary',
  'GICS Sub-Industry': 'Broadline Retail',
  'date_founded': 1994,
  'date_added': '2005-11-18'},
 {'Security': 'Apple Inc.',
  'GICS Sector': 'Information Technology',
  'GICS Sub-Industry': 'Technology Hardware, Storage & Peripherals',
  'date_founded': 1977,
  'date_added': '1982-11-30'},
 {'Security': 'Arthur J. Gallagher & Co.',
  'GICS Sector': 'Financials',
  'GI

### Add data to collection

In [19]:
collection.add(
     ids = ids,
     documents=documents,
     metadatas=metadata
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 91.9MiB/s]


In [20]:
collection.count()

48

### Query data

**query_texts**: input in text format on
which we want to find similar vectors.
query_embeddings: input in vector format over which we want to find similar vectors.

**n_results:** Number of results to be returned by the search.

**where:** Filter vectors based on metadata.

**where_document:** Filter vectors based on which documents contain specific content.

**include:** what the search should return. By default it returns all available information: (“metadatas”, “documents”, “distances”).

let’s extract the documents that most closely resemble the prompt phone manufacturer:


In [21]:
results = collection.query(
    query_texts="phone manufacturers",
    n_results=5
)
results

{'ids': [['11', '1', '2', '13', '25']],
 'embeddings': None,
 'documents': [['Caterpillar Inc. is an American construction, mining and other engineering equipment manufacturer. The company is the world\'s largest manufacturer of construction equipment.\nIn 2018, Caterpillar was ranked number 73 on the Fortune 500 list and number 265 on the Global Fortune 500 list. Caterpillar stock is a component of the Dow Jones Industrial Average.\nCaterpillar Inc. traces its origins to the 1925 merger of the Holt Manufacturing Company and the C. L. Best Tractor Company, creating a new entity,  California-based Caterpillar Tractor Company. In 1986, the company reorganized itself as a Delaware corporation under the current name, Caterpillar Inc. It announced in January 2017 that over the course of that year, it would relocate its headquarters from Peoria, Illinois, to Deerfield, Illinois, scrapping plans from 2015 of building an $800 million new headquarters complex in downtown Peoria. Its headquarter

Filter the results to only include companies whose GICS Sector is Information Technology.

In [22]:
collection.query(
    query_texts="phone manufacturers",
    n_results=5,
    where = {'GICS Sector': 'Information Technology'}
)

{'ids': [['24', '4', '34', '18', '0']],
 'embeddings': None,
 'documents': [["HP Inc. is an American multinational information technology company with its headquarters in Palo Alto, California, that develops personal computers (PCs), printers and related supplies, as well as 3D printing services. It is the world's second-largest personal computer vendor by unit sales after Lenovo and ahead of Dell as of 2024.\nHP Inc. was founded in 2015 when the original Hewlett-Packard Company split into two companies. The old company's enterprise product and business services divisions were spun-off into a new publicly traded company, Hewlett Packard Enterprise. At the same time as the divesture, Hewlett-Packard Company renamed itself to HP Inc. and retained the personal computer and printer services divisions of its predecessor, serving as the legal successor of the original company that was founded in 1939. HP is listed on the New York Stock Exchange and is a constituent of the S&P 500 Index. In t

# more filters

$eq: equal to.

$ne: not equal to.

$gt: greater than.

$gte: greater than or equal to.

$lt: less than.

$lte: less than or equal to.

Let’s apply the same query to companies in the Information Technology sector but, in addition, their creation date is greater than 1990.

In [23]:
where_clause = {
    "$and" : [
        {"GICS Sector" : {"$eq": "Information Technology"}},
        {"date_founded": {"$gte": 1990}}
    ]
}

results = collection.query(
    query_texts="phone manufacturers",
    n_results=5,
    where = where_clause
)



In [24]:
results

{'ids': [['18', '46']],
 'embeddings': None,
 'documents': [["F5, Inc. is an American technology company specializing in application security, multi-cloud management, online fraud prevention, application delivery networking (ADN), application availability and performance, and network security, access, and authorization.\nF5 originally offered application delivery controller (ADC) technology, but has since expanded into application layer, automation, multi-cloud, and security services. As ransomware, data leaks, DDoS, and other attacks on businesses of all sizes are arising, companies such as F5 have continued to reinvent themselves.\nF5 is headquartered in Seattle, Washington in F5 Tower, with an additional 75 offices in 43 countries focusing on account management, global services support, product development, manufacturing, software engineering, and administrative jobs. Notable office locations include Spokane, Washington; New York, New York; Boulder, Colorado; London, England; San Jo

In [25]:
[(el['Security'], el['GICS Sector'], el['date_founded']) for el in results['metadatas'][0]]

[('F5, Inc.', 'Information Technology', 1996),
 ('Workday, Inc.', 'Information Technology', 2005)]